In [ ]:
from scatspectra import PriceData, MRWGenerator
import numpy as np
import matplotlib.pyplot as plt

from scipy.stats import kurtosis, skew, norm, expon, cauchy

# Reminders on kurtosis and skewness
## Kurtosis
Kurtosis is a measure of the "tailedness" of the probability distribution of a real-valued random variable. It is a descriptor of the shape of a probability distribution and provides information about the tails of the distribution.
It requires the calculation of the fourth central moment.
$$\beta_2 = \frac{\mu_4}{\sigma^4}=\mathbb E\left[\left(\frac{X-\mu}{\sigma}\right)^4\right]$$
where $\mu_4$ is the fourth central moment and $\sigma$ is the standard deviation.
## Skewness
Skewness is a measure of the asymmetry of the probability distribution of a real-valued random variable about its mean. The skewness value can be positive, zero, negative, or undefined.
It requires the calculation of the third central moment.
$$\gamma_1 = \frac{\mu_3}{\sigma^3}=\mathbb E\left[\left(\frac{X-\mu}{\sigma}\right)^3\right]$$
where $\mu_3$ is the third central moment and $\sigma$ is the standard deviation.
## Relationship between kurtosis and skewness
The kurtosis is related to the skewness by the following formula:
$$\beta_2 = \gamma_1^2 + 1$$
## Kurtosis and skewness of some distributions
- Normal distribution: $\beta_2=3$ and $\gamma_1=0$
- Uniform distribution: $\beta_2=-\frac{6}{5}$ and $\gamma_1=0$
- Exponential distribution: $\beta_2=9$ and $\gamma_1=2$
- Cauchy distribution: $\beta_2=\infty$ and $\gamma_1=\infty$

## Visualizing kurtosis and skewness

In [ ]:
normal_sample = np.random.normal(0, 1, 10_000)
plt.figure(figsize=(10, 5))
plt.hist(normal_sample, density=True, alpha=0.5, label="Normal Distribution", bins=100)
plt.title("Histogram of normal distribution")
plt.xlabel("Value")
plt.ylabel("Frequency")
plt.plot(
    np.linspace(np.min(normal_sample), np.max(normal_sample), 100),
    norm.pdf(np.linspace(np.min(normal_sample), np.max(normal_sample), 100)),
    label="PDF",
    color="black",
)
plt.legend(loc="upper right")
plt.axis(xmin=np.min(normal_sample), xmax=np.max(normal_sample))
plt.show()
print("Kurtosis of normal distribution: ", kurtosis(normal_sample, fisher=False))
print("Skewness of normal distribution: ", skew(normal_sample))

In [ ]:
expo_sample = np.random.exponential(1, 10_000)
plt.figure(figsize=(10, 5))
plt.hist(expo_sample, density=True, alpha=0.5, label="Exponential Distribution", bins=100)
plt.title("Histogram of exponential distribution")
plt.xlabel("Value")
plt.ylabel("Frequency")
plt.plot(
    np.linspace(np.min(expo_sample), np.max(expo_sample), 100),
    expon.pdf(np.linspace(np.min(expo_sample), np.max(expo_sample), 100)),
    label="PDF",
    color="black",
)
plt.legend(loc="upper right")
plt.axis(xmin=np.min(expo_sample), xmax=np.max(expo_sample))
plt.show()
print("Kurtosis of exponential distribution: ", kurtosis(expo_sample, fisher=False))
print("Skewness of exponential distribution: ", skew(expo_sample))